- Notebook based on "postprocessing_small_test_run.ipynb", copied and modified
- File: ./Output/from_server/27-05-2021_14.58_full.txt (resp. -COPY.txt with last line (containing terminal call, see right below) is removed
- Parameters: ['--dict', './Lexicons/childlex_0.17.01_all_types.txt', '--train', './Trainingdata/h1obi_train_misspellings.csv', '--test', './Input_Server/2021-04-14_litkey_2_error_types_cs_count.tsv', '--window', '3', '--minatoa', '0.8']

- <span style="color:green"> Fixed: For loops do not run through </span>
    - <span style="color:green"> Details: Reimplement! For now, outsource on server: _postprocessing-for-loop-server.py_  </span>
    - <span style="color:green"> Fix is: Use merge </span>

# Outline
1. Read in results of Boyd spell-checker and transform it to dataframe that has known format (original, corrected, suggestions)
    - i. e. type-base
2. Reconstruct dataframe to put into evaluate-function
    - prep
    - get filenames back
        - type base
        - token base
3. Evaluate
4. Export

In [1]:
import pandas as pd
import helpers_boyd

import sys
sys.path.insert(0, '/Users/lisaprepens/coding/bodu-spell/spelling_correction')
from Lisa import litkey_2

# Do not truncate columns of DataFrame
pd.set_option('display.max_rows', None)

# 1 - Read in results of Boyd spell-checker

In [2]:
data = pd.read_csv(r'./Output/from_server/05-11-2022_19.49_FULL_HUN_DICT-COPY.txt', delimiter='\t', header=None)

In [3]:
data.head(10)
data.tail(10)

,0,1,2,3,4,5,6,7,8,9,...,13,14,15,16,17,18,19,20,21,22
9474,hette,hätte,1,hätte,1.75,hetzte,3.72,hätten,4.78,hättet,...,hechtete,5.95,hetzten,6.73,hetze,7.09,hetztet,7.10,heckte,7.30
9475,Telefen,Telefon,1,Telefon,5.59,Telefonen,7.79,Telefone,8.53,Telefons,...,Tiefen,12.07,Telefonie,12.99,Teerseifen,13.32,Telefonat,13.84,Telefonieren,14.04
9476,aufzumunter,aufzumuntern,1,aufzumuntern,3.15,aufzumunternd,7.32,aufzumunternder,10.23,aufzumunternde,...,aufzumunternden,11.24,aufzumunterndes,13.67,aufzumauernder,14.69,auszumustern,15.91,anzumuten,17.56
9477,Ungefär,Ungefähr,1,ungefähr,5.17,ungefähre,7.93,ungefähren,8.00,ungefährem,...,ungefärbt,10.77,ungefährlich,11.45,ungefähres,11.53,ungefärbtes,11.82,ungefärbte,13.57
9478,nooh,noch,1,noch,8.41,Noah,9.06,Note,11.06,nottäte,...,oho,13.09,knote,13.73,roter,13.74,NaOH,13.87,Noten,14.08
9479,nieht,nicht,2,niet,5.67,nicht,5.74,niest,7.61,näht,...,sieht,8.10,zieht,8.53,lieht,8.61,nichts,8.90,riecht,8.95
9480,vessen,fassen,1,wessen,5.80,fesseln,6.41,fressen,7.53,Vesser,...,Fesseln,8.41,vergessen,8.67,vernässen,8.74,dessen,8.90,fassen,8.92
9481,Tor,Tür,1,Tor,0.17,Torr,2.06,Tors,2.87,Thor,...,Tore,3.62,Toren,4.09,Tour,4.10,Tenor,4.33,Tori,4.95
9482,mutzte,nutzte,1,nutzte,5.87,motzte,6.24,benutzte,7.15,muckste,...,nützte,8.12,nutzten,8.93,putzte,9.23,motzten,9.29,nutztet,9.29
9483,momen,Moment,1,Moment,4.99,Nomen,5.99,Mommsen,6.32,Momentes,...,mopsen,7.65,Momenten,7.87,Momente,8.08,myopen,8.45,Normen,8.64


In [4]:
data_ready = helpers_boyd.get_suggestions_list(data)

In [5]:
display(data_ready.head(2))
display(data_ready.shape[0])

,original,corrected,suggestions
0,belt,bellt,"[ballt, bellt, Belt, bellst, hebelt, behelmt, ..."
1,kukt,kuckt,"[kickt, guckt, guckst, kickst, kickte, gurkt, ..."


9484

# 2 - Reconstruct dataframe
_given: type-base with suggestions_

0) Prep

a) get filename back (for type and token base)

## 0) Prep

In [6]:
# Get information back
# Read in dataframe to merge with
types = litkey_2.load(litkey_data_path = "../../litkey-data/")
token = litkey_2.load(litkey_data_path = "../../litkey-data/", toss_duplicates=False)

## a) get filename back (type and token base)
- both times given: "original" DFs that go into analysis (after preprocessing), i. e. type- and token-base
- input to spell-checker is only value of original-column, meaning all originals are given the same suggestions
- _**iterate over DF and compare 'originals', glue boyd-suggestions onto that**_

In [17]:
# Add empty suggestions column
# When using at method, dtype of column has to be object

#token['suggestions']=''
#token['suggestions'] = token['suggestions'].astype('object')
#display(token.head(2))
#display(token.shape[0])

#types['suggestions']=''
#types['suggestions'] = types['suggestions'].astype('object')
#display(types.head(2))
#display(types.shape[0])

,original,corrected,filename,freq_ori,freq_cor,freq_tup,suggestions
0,belt,bellt,01-005-2-III-Eis.csv,91,138,91,
1,kukt,kuckt,01-005-2-III-Eis.csv,73,152,73,


24601

,original,corrected,filename,freq_ori,freq_cor,freq_tup,suggestions
0,belt,bellt,01-005-2-III-Eis.csv,91,138,91,
1,kukt,kuckt,01-005-2-III-Eis.csv,73,152,73,


9484

In [7]:
# Export pickle to run on server
import pickle
print(pickle.HIGHEST_PROTOCOL)
print("old",pickle.DEFAULT_PROTOCOL)
pickle.DEFAULT_PROTOCOL = 3
print("new",pickle.DEFAULT_PROTOCOL)

token.to_pickle("./outsourcing_postpr_for_loop/input_data_error_token_boyd_server.pkl", protocol=3)
types.to_pickle("./outsourcing_postpr_for_loop/input_data_error_types_boyd_server.pkl", protocol=3)
data_ready.to_pickle("./outsourcing_postpr_for_loop/boyd_output_suggestions_list.pkl", protocol=3)

5
old 4
new 3


FileNotFoundError: [Errno 2] No such file or directory: './outsourcing_postpr_for_loop/input_data_error_token_boyd_server.pkl'

---

In [8]:
# Merge
types_m = pd.merge(types, data_ready, on=['original', 'corrected'])
token_m = pd.merge(token, data_ready, on=['original', 'corrected'])

In [9]:
print(types_m.shape[0])
print(token_m.shape[0])

9484
24601


In [13]:
token_m.head(100)
types_m.tail(20)

,original,corrected,filename,freq_ori,freq_cor,freq_tup,suggestions
9464,spruch,sprach,10-670-4-IV-Weg.csv,1,1,1,"[Spruch, sprüh, Spruchs, sprühst, sprach, spri..."
9465,bemerkde,bemerkte,10-682-4-IV-Weg.csv,1,6,1,"[bemerkte, bemerkende, bemerkten, bemerktes, b..."
9466,hängde,hängte,10-682-4-IV-Weg.csv,1,23,1,"[hängende, hängte, hängenden, hängendes, hänge..."
9467,Kugde,kuckte,10-682-4-IV-Weg.csv,1,43,1,"[Kurde, Kunde, guckende, Kurden, Kunigunde, Kü..."
9468,bedankde,bedankte,10-682-4-IV-Weg.csv,1,5,1,"[bedankende, bedankte, bedankenden, bedankende..."
9469,LoLie,Lolli,10-682-4-IV-Weg.csv,1,259,1,"[Lolas, Lobelie, Loblied, Lilie, Lobliedern, L..."
9470,stockchen,Stöckchen,10-692-4-IV-Weg.csv,1,1,1,"[Stöckchen, stocken, stockten, Stückchen, Stoc..."
9471,Bielder,Bilder,10-692-4-IV-Weg.csv,1,8,1,"[Bilder, Bildern, Bildner, Bildes, Bilden, Bil..."
9472,ieh,ich,10-693-4-IV-Weg.csv,1,11,1,"[lieh, sieh, zieh, ich, siehe, hiev, siech, jä..."
9473,aufgeh.passt,aufgepasst,10-693-4-IV-Weg.csv,1,4,1,"['s, ABC, ABM, ACLs, ACL, ACPI, ADAC, ADGB-Vor..."


---

In [35]:
#def match_corrected(df_token, df_sugg):
#    for i in range(len(df_token)):
#        #print(i)
#        if df_token.iloc[i].corrected == df_sugg.iloc[i].corrected:
#            print(df_token.iloc[i].corrected, df_sugg.iloc[i].suggestions)
#            #df_token.loc['suggestions'] = data_token.suggestions.iloc[].index(data) 

In [ ]:
# TOKEN
# looping over df is non-pythonic; better write function with if else statement and apply to df...
# Note: For the whole data set, it takes reaaaaaaally long; Over 5 hours?!
for index, row in token.iterrows():
        for index_2, row_2 in data_ready.iterrows():
        
            if row.original == row_2.original:
                
                #print(index, index_2)
                #print(row.corrected, row_2.suggestions)
                # Loc does not work when adding list to dataframe cell
                #test_token_100.loc[row_2, 'suggestions'] = row_2.suggestions
                token.at[index, 'suggestions'] = row_2.suggestions

In [ ]:
# TYPES
# looping over df is non-pythonic; better write function with if else statement and apply to df...
for index, row in types.iterrows():
        for index_2, row_2 in data_ready.iterrows():
        
            if row.original == row_2.original:
                
                #print(index, index_2)
                #print(row.corrected, row_2.suggestions)
                # Loc does not work when adding list to dataframe cell
                #test_token_100.loc[row_2, 'suggestions'] = row_2.suggestions
                types.at[index, 'suggestions'] = row_2.suggestions

In [18]:
#display(token.head(20))
#display(token.tail(20))

,original,corrected,filename,freq_ori,freq_cor,freq_tup,suggestions
0,belt,bellt,01-005-2-III-Eis.csv,91,138,91,"[bellt, hebelt, pellt, bellten, bellte, perlt,..."
1,kukt,kuckt,01-005-2-III-Eis.csv,73,152,73,"[kuckt, guckt, guckten, guckte, guckst, gucken..."
2,dan,dann,01-005-2-III-Eis.csv,627,651,621,"[Dan, dann, Dann, dran, dank, den, Dean, danne..."
3,gekricht,gekriegt,01-005-2-III-Eis.csv,2,15,2,"[gekreischt, gekracht, gekriegt, Gericht, gebr..."
4,leken,lecken,01-005-2-III-Eis.csv,14,17,14,"[lecken, Lecken, lenken, legen, leckten, bleck..."
5,felt,fällt,01-005-2-III-Eis.csv,93,198,90,"[fällt, fehlt, Fehlt, Fällt, feilt, Feld, fäll..."
6,wolte,wollte,01-005-2-III-Eis.csv,173,201,173,"[wollte, Wollte, Volte, wollten, wolltet, wohn..."
7,lekt,leckt,01-005-2-III-Eis.csv,20,42,19,"[leckt, legt, Leckt, lenkt, Legt, bleckt, leck..."
8,fom,vom,01-005-2-III-Eis.csv,13,16,13,"[vom, vorm, Form, formt, formen, fromm, Formen..."
9,gawen,kaufen,01-006-2-III-Eis.csv,1,6,1,"[geruhen, genauen, gehauen, Gaumen, grauen, ge..."


In [ ]:
display(types.head(20))
display(types.tail(20))

---

# 3 - Evaluate

In [14]:
# Evaluate
# Token
eval_token_lc = litkey_2.evaluate(token_m)
eval_token_cs = litkey_2.evaluate(token_m, lower_case=False)

  0%|          | 0/24601 [00:00<?, ?it/s]

  0%|          | 0/24601 [00:00<?, ?it/s]

In [15]:
# Types
eval_types_lc = litkey_2.evaluate(types_m)
eval_types_cs = litkey_2.evaluate(types_m, lower_case=False)

  0%|          | 0/9484 [00:00<?, ?it/s]

  0%|          | 0/9484 [00:00<?, ?it/s]

In [16]:
display(eval_token_cs.head(10))
display(eval_token_lc.head(10))

display(eval_types_cs.head(10))
display(eval_types_lc.head(10))

,original,corrected,filename,freq_ori,freq_cor,freq_tup,suggestions,in_sugg,sugg_idx,idx0
0,belt,bellt,01-005-2-III-Eis.csv,91,138,91,"[ballt, bellt, Belt, bellst, hebelt, behelmt, ...",True,1.0,False
1,belt,bellt,01-338-2-III-Eis.csv,91,138,91,"[ballt, bellt, Belt, bellst, hebelt, behelmt, ...",True,1.0,False
2,belt,bellt,01-355-2-III-Eis.csv,91,138,91,"[ballt, bellt, Belt, bellst, hebelt, behelmt, ...",True,1.0,False
3,belt,bellt,02-220-2-IV-Weg.csv,91,138,91,"[ballt, bellt, Belt, bellst, hebelt, behelmt, ...",True,1.0,False
4,belt,bellt,02-443-2-IV-Weg.csv,91,138,91,"[ballt, bellt, Belt, bellst, hebelt, behelmt, ...",True,1.0,False
5,belt,bellt,02-468-2-IV-Weg.csv,91,138,91,"[ballt, bellt, Belt, bellst, hebelt, behelmt, ...",True,1.0,False
6,belt,bellt,03-291-3-I-Frosch.csv,91,138,91,"[ballt, bellt, Belt, bellst, hebelt, behelmt, ...",True,1.0,False
7,belt,bellt,03-294-3-I-Frosch.csv,91,138,91,"[ballt, bellt, Belt, bellst, hebelt, behelmt, ...",True,1.0,False
8,belt,bellt,03-329-3-I-Frosch.csv,91,138,91,"[ballt, bellt, Belt, bellst, hebelt, behelmt, ...",True,1.0,False
9,belt,bellt,03-329-3-I-Frosch.csv,91,138,91,"[ballt, bellt, Belt, bellst, hebelt, behelmt, ...",True,1.0,False


,original,corrected,filename,freq_ori,freq_cor,freq_tup,suggestions,in_sugg,sugg_idx,idx0
0,belt,bellt,01-005-2-III-Eis.csv,91,138,91,"[ballt, bellt, belt, bellst, hebelt, behelmt, ...",True,1.0,False
1,belt,bellt,01-338-2-III-Eis.csv,91,138,91,"[ballt, bellt, belt, bellst, hebelt, behelmt, ...",True,1.0,False
2,belt,bellt,01-355-2-III-Eis.csv,91,138,91,"[ballt, bellt, belt, bellst, hebelt, behelmt, ...",True,1.0,False
3,belt,bellt,02-220-2-IV-Weg.csv,91,138,91,"[ballt, bellt, belt, bellst, hebelt, behelmt, ...",True,1.0,False
4,belt,bellt,02-443-2-IV-Weg.csv,91,138,91,"[ballt, bellt, belt, bellst, hebelt, behelmt, ...",True,1.0,False
5,belt,bellt,02-468-2-IV-Weg.csv,91,138,91,"[ballt, bellt, belt, bellst, hebelt, behelmt, ...",True,1.0,False
6,belt,bellt,03-291-3-I-Frosch.csv,91,138,91,"[ballt, bellt, belt, bellst, hebelt, behelmt, ...",True,1.0,False
7,belt,bellt,03-294-3-I-Frosch.csv,91,138,91,"[ballt, bellt, belt, bellst, hebelt, behelmt, ...",True,1.0,False
8,belt,bellt,03-329-3-I-Frosch.csv,91,138,91,"[ballt, bellt, belt, bellst, hebelt, behelmt, ...",True,1.0,False
9,belt,bellt,03-329-3-I-Frosch.csv,91,138,91,"[ballt, bellt, belt, bellst, hebelt, behelmt, ...",True,1.0,False


,original,corrected,filename,freq_ori,freq_cor,freq_tup,suggestions,in_sugg,sugg_idx,idx0
0,belt,bellt,01-005-2-III-Eis.csv,91,138,91,"[ballt, bellt, Belt, bellst, hebelt, behelmt, ...",True,1.0,False
1,kukt,kuckt,01-005-2-III-Eis.csv,73,152,73,"[kickt, guckt, guckst, kickst, kickte, gurkt, ...",False,NaN,False
2,dan,dann,01-005-2-III-Eis.csv,627,651,621,"[dann, darin, Dahn, dank, dran, den, dar, Jan,...",True,0.0,True
3,gekricht,gekriegt,01-005-2-III-Eis.csv,2,15,2,"[gekriegt, gekracht, gekreischt, verkriecht, g...",True,0.0,True
4,leken,lecken,01-005-2-III-Eis.csv,14,17,14,"[lecken, lenken, legen, Lecken, leckem, lecken...",True,0.0,True
5,felt,fällt,01-005-2-III-Eis.csv,93,198,90,"[fallt, fehlt, fällt, feilt, fackelt, verhält,...",True,2.0,False
6,wolte,wollte,01-005-2-III-Eis.csv,173,201,173,"[wollte, Volte, wohlste, wollten, wolltet, wol...",True,0.0,True
7,lekt,leckt,01-005-2-III-Eis.csv,20,42,19,"[leckst, leckt, legt, lenkt, legst, lägt, lenk...",True,1.0,False
8,fom,vom,01-005-2-III-Eis.csv,13,16,13,"[vom, vorn, vorm, Form, fromm, formt, firm, fo...",True,0.0,True
9,gawen,kaufen,01-006-2-III-Eis.csv,1,6,1,"[Gauen, grauen, gehauen, genauen, kauen, geruh...",False,NaN,False


,original,corrected,filename,freq_ori,freq_cor,freq_tup,suggestions,in_sugg,sugg_idx,idx0
0,belt,bellt,01-005-2-III-Eis.csv,91,138,91,"[ballt, bellt, belt, bellst, hebelt, behelmt, ...",True,1.0,False
1,kukt,kuckt,01-005-2-III-Eis.csv,73,152,73,"[kickt, guckt, guckst, kickst, kickte, gurkt, ...",False,NaN,False
2,dan,dann,01-005-2-III-Eis.csv,627,651,621,"[dann, darin, dahn, dank, dran, den, dar, jan,...",True,0.0,True
3,gekricht,gekriegt,01-005-2-III-Eis.csv,2,15,2,"[gekriegt, gekracht, gekreischt, verkriecht, g...",True,0.0,True
4,leken,lecken,01-005-2-III-Eis.csv,14,17,14,"[lecken, lenken, legen, lecken, leckem, lecken...",True,0.0,True
5,felt,fällt,01-005-2-III-Eis.csv,93,198,90,"[fallt, fehlt, fällt, feilt, fackelt, verhält,...",True,2.0,False
6,wolte,wollte,01-005-2-III-Eis.csv,173,201,173,"[wollte, volte, wohlste, wollten, wolltet, wol...",True,0.0,True
7,lekt,leckt,01-005-2-III-Eis.csv,20,42,19,"[leckst, leckt, legt, lenkt, legst, lägt, lenk...",True,1.0,False
8,fom,vom,01-005-2-III-Eis.csv,13,16,13,"[vom, vorn, vorm, form, fromm, formt, firm, fo...",True,0.0,True
9,gawen,kaufen,01-006-2-III-Eis.csv,1,6,1,"[gauen, grauen, gehauen, genauen, kauen, geruh...",False,NaN,False


# 4 - Export

In [17]:
# Export
eval_token_cs.to_pickle("data_error_token_boyd_evaluation_cs_hun-dict.pkl")
eval_types_cs.to_pickle("data_error_types_boyd_evaluation_cs_hun-dict.pkl")

eval_token_lc.to_pickle("data_error_token_boyd_evaluation_lc_hun-dict.pkl")
eval_types_lc.to_pickle("data_error_types_boyd_evaluation_lc_hun-dict.pkl")